In [ ]:
!pip install generative-ai-hub-sdk

In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
# Load PDF
loaders = [
    PyPDFLoader("lauchpad.pdf"),
    PyPDFLoader("core.pdf")]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 100
)

In [ ]:
splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

In [ ]:
from gen_ai_hub.proxy.langchain.init_models import init_llm
from gen_ai_hub.proxy.langchain.init_models import init_embedding_model


llm = init_llm('gpt-4', temperature=0., max_tokens=256)
embeddings = init_embedding_model('text-embedding-ada-002')

In [ ]:
from langchain.vectorstores.pgvector import PGVector

In [ ]:
# PGVector needs the connection string to the database.
CONNECTION_STRING = "postgresql+psycopg2://<username>:<password>@<Host>:5898/<dbname>"

In [ ]:
COLLECTION_NAME = "ai-docs"

db = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

In [ ]:
query = "how to deploy a model in gen ai hub"

In [ ]:
results = db.similarity_search(query, k=1)
results

In [ ]:
answer = results[0].page_content
answer

In [ ]:
llm.predict("Refer this "+ answer + " and now answer " + query)

In [ ]:
docs_with_score = db.similarity_search_with_score(query)
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

In [ ]:
docs_with_score = db.max_marginal_relevance_search_with_score(query)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

